# コンテナを利用して Lambda 関数を作成する
このスクリプトを実行するための IAM の権限や、AWS configure とかは事前に要設定

In [ ]:
import json
import pprint

with open('conf.json') as f:
    conf = json.load(f)

pprint.pprint(conf)

In [ ]:
import boto3, datetime, subprocess, json
from time import sleep

from boto3.session import Session
profile = conf["profile"]
session = Session(profile_name=profile)

In [ ]:
# 様々な名前をユニークにするため、現在時刻のyyyymmddhhmmssを利用する
now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9))).strftime('%Y%m%d%H%M%S')
image = 'lambda-container-yolov5'
tag = ':latest'
repository_name = f'{image}-{now}'

## ECR リポジトリを boto3 で作成

In [ ]:
%%time
# ECR のリポジトリを作成する
ecr = session.client('ecr')
response = ecr.create_repository(
    repositoryName=repository_name,
    imageScanningConfiguration={'scanOnPush': True},
)

In [ ]:
# 必要な情報を抜き取っておく
uri = response['repository']['repositoryUri']
account_id = response['repository']['registryId']
region = uri.split('.')[3]
domain = uri.split('/')[0]

## ECR にコンテナイメージをプッシュ

In [ ]:
%%time

# build からプッシュまで

# build 済なら実行不要
!docker build -t {repository_name} --platform=linux/amd64 .

# タグ付与
!docker tag {repository_name}{tag} {uri}{tag}

# ECR にログイン
! aws ecr get-login-password --profile {profile} | docker login --username AWS --password-stdin {domain}

# push
!docker push {uri}{tag}

In [ ]:
# 必要な情報を抜き取っておく
res = ecr.describe_images(
    repositoryName = repository_name
)
image_digest = res['imageDetails'][-1]['imageDigest']

## Lambda を実行するロールを設定
1. ロール作成
2. ポリシー作成
3. ポリシーをロールにアタッチ

In [ ]:
iam = session.client('iam')
function_name = f'{image}-function-{now}'
doc = {
    'Version': '2012-10-17',
    'Statement': [
        {
            'Action': 'sts:AssumeRole',
            'Principal': {
                'Service': 'lambda.amazonaws.com'
                
            },
            'Effect': 'Allow',
            'Sid': ''
            
        }
    ]
}

# ロール作成
role_name = f'{image}-role-{now}'
res = iam.create_role(
    Path = '/service-role/',
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(doc),
    Description=f'exec role',
    MaxSessionDuration=3600*12
)
role_arn = res['Role']['Arn']

doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "logs:CreateLogGroup",
            "Resource": f"arn:aws:logs:{account_id}:*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                f"arn:aws:logs:{region}:{account_id}:log-group:/aws/lambda/{function_name}:*"
            ]
        }
    ]
}

# ポリシー作成
poicy_name = f'{image}-policy-{now}'
res = iam.create_policy(
    PolicyName=poicy_name,
    PolicyDocument=json.dumps(doc),
)
policy_arn = res['Policy']['Arn']

# 作成したポリシーをロールにアタッチ
res = iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)

# IAM の反映をしばし待つ
sleep(20)

## Lambda 関数を作成する
IAM の反映が間に合わない場合があるので、 sleep を入れているが、十分に時間が経っている場合は不要

In [ ]:
%%time
# lambda function を Docker で作成したイメージから作成する
lambda_client = session.client('lambda')
res = lambda_client.create_function(
    FunctionName=function_name,
    Role=role_arn,
    Code={
        'ImageUri': f'{uri}@{image_digest}'
    },
    Description='input-> b64img, output -> b64img, yolov5 detect',
    Timeout=60*15,
    MemorySize=1024,
    Publish=True,
    PackageType='Image',
)
# 作成が完了するまで待つ
while True:
    res = lambda_client.get_function(FunctionName=function_name)
    try:
        if res['Configuration']['StateReasonCode']=='Creating':
            print('.',end='')
            sleep(1)
    except:
        if res['Configuration']['LastUpdateStatus']=='Successful':
            print('!')
            break
        else:
            print('?')
            break


## 推論してみる

In [ ]:
import base64
import requests
import cv2
import numpy as np
from matplotlib import pyplot as plt
input_file = './yolov5/data/images/bus.jpg'
data = {}
with open(input_file,'rb') as f:
    data['img']= base64.b64encode(f.read()).decode('utf-8')

In [ ]:
%%time
res = lambda_client.invoke(
    FunctionName=function_name,
    Payload=json.dumps(data)
)

payload = res["Payload"]

body = payload.read()
body = body.decode()
json_dict = json.loads(body)
print(json_dict)

## お片付け

In [ ]:
lambda_client.delete_function(
    FunctionName=function_name
)

In [ ]:
iam.detach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)

In [ ]:
iam.delete_policy(
    PolicyArn=policy_arn
)

In [ ]:
iam.delete_role(
    RoleName=role_name,
)

In [ ]:
ecr.batch_delete_image(
    repositoryName=repository_name,
    imageIds=[{
        'imageDigest':image_digest
    }]
)

In [ ]:
ecr.delete_repository(
    repositoryName=repository_name,
)